### 第一步：从"市场"导入预置的数据集
目前ModelArts SDK/API暂时不支持对"市场"的操作，在本地使用OBS SDK上传数据；或使用web console导入数据
(ModelArts notebook中无法访问网络，只能通过web console操作导入数据)

In [1]:
## 准备SDK
!pip install esdk-obs-python

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
%env USER_ACCESS_KEY=my_ak
%env USER_SECRET_ACCESS_KEY=my_sk
%env PROJECT_ID=my_projid

env: USER_ACCESS_KEY=SC36F1SPFVDDCD8MDFNG
env: USER_SECRET_ACCESS_KEY=V5MEsWv9ws8aVEgepEaflE01Tow5ybdRZUoZagIp
env: PROJECT_ID=cf1bed56cdc045b196278cace96852a8


In [3]:
import os
from obs import ObsClient

AK=os.getenv('USER_ACCESS_KEY')
SK=os.getenv('USER_SECRET_ACCESS_KEY')
PROJECT_ID=os.getenv('PROJECT_ID')

obs_endpoint=os.getenv('MINER_OBS_URL')
if obs_endpoint is None:
    obs_endpoint='obs.cn-north-1.myhuaweicloud.com'


obsClient = ObsClient(
    access_key_id=AK,
    secret_access_key=SK,
    server=obs_endpoint
)

In [4]:
TARGET_BUCKET = 'ma-workflow'
resp = obsClient.headBucket(TARGET_BUCKET)
if resp.status == 404 and obsClient.createBucket(TARGET_BUCKET).status >= 300:
    print("Failed to create bucket{}".format(TARGET_BUCKET))


In [5]:
resp = obsClient.putFile(TARGET_BUCKET, 'mnist/code/train_mnist.py', file_path='./train_mnist.py')
print(resp.body)
if resp.status >= 300:
    print('Failed to put file')  

{'versionId': 'G0011168E7158199FFFF804E0F98FFB8', 'etag': '"5fdf6bbc3ef5d5123a31fd6f1a6055b0"', 'objectUrl': 'https://ma-workflow.obs.cn-north-1.myhuaweicloud.com/mnist/code/train_mnist.py'}


创建ModelArts客户端，注意需要在HOME目录下配置.modelarts/config.json
详情参见[初始化ModelArts SDK](https://support.huaweicloud.com/sdkreference-modelarts/modelarts_04_0006.html)
注意：非企业用户account为空，只填username

In [6]:
from modelarts import config
from modelarts.client.api import *
from modelarts.client.rest import ApiException
maClient = config.create_client(context='default')

E:\Dev Tools\Anaconda3\envs\py36\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


创建训练作业
- 参考ModelArts SDK说明，body与本目录下training_job.json一致
- 对于文档中参数说明不明确之处，在web console中创建训练任务并观察请求参数，request body保存在sample_training_job.json中
- spec_id的获取参考文档 [查看作业资源规格](https://support.huaweicloud.com/sdkreference-modelarts/modelarts_04_0074.html)
- engine_id的获取参考文档 [查看作业引擎规格](https://support.huaweicloud.com/sdkreference-modelarts/modelarts_04_0075.html)


In [8]:
# 注意事项xx_url应该是目录，必须以/结尾
# engine_id=29: MXNet-1.2.1-python3.6
# model_id优先级高于app_url & boot_file_url，若指定model_id则优先从已有模型中读取参数并进行训练
import json
training_job_body = {
  "job_name": "mxnet-mnist",
  "job_desc": "mxnet-mnist with ModelArts SDK",
  "config": {
    "worker_server_num": 1,
#     "model_id": 3,
    "app_url": "/ma-workflow/mnist/code/",
    "boot_file_url": "/ma-workflow/mnist/code/train_mnist.py",
    "data_source": [
      {
        "type": "obs",
        "data_url": "/ma-workflow/mnist/"
      }
    ],
    "train_url": "/ma-workflow/mnist/output/",
    "log_url": "/ma-workflow/mnist/log/",
    "spec_id": 2,
    "engine_id": 29
  }
}

trainJobApi = TrainJobApi(maClient)
try:
    training_job = trainJobApi.create_training_job(project_id=PROJECT_ID, body=training_job_body)
    job_id = training_job.job_id
    version_id = training_job.version_id
except ApiException as e:
    resp_obj = json.loads(e.body)
    print(e)
    if resp_obj['error_code'] == 'ModelArts.0103': # 重名，作业已存在，创建作业版本
        # 当前SDK需要先列出job list，再按照job_name查找到job_id，才能创建作业版本
        # 删除job也需要job_id
        print("Let's createa job version, it's a pretty tough task for now, we'll do it later!")
        
    
        

(400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 13 Feb 2019 13:40:25 GMT', 'Content-Type': 'application/json', 'Content-Length': '116', 'Connection': 'keep-alive', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'X-Frame-Options': 'DENY', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Content-Security-Policy': "default-src 'self'", 'X-Permitted-Cross-Domain-Policies': 'master-only', 'Server': 'api-gateway', 'X-Request-Id': '34191ee714a54ea2407cdb0f579475f0'})
HTTP response body: {"is_success":false,"error_code":"ModelArts.0103","error_message":"Job name:mxnet-mnist is existed","error_info":{}}

Let's createa job version, it's a pretty tough task for now, we'll do it later!


E:\Dev Tools\Anaconda3\envs\py36\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


监听训练任务的状态

In [9]:
# info = trainJobApi.view_training_job(project_id=PROJECT_ID, job_id=job_id, version_id=version_id)
# print(info)
from time import sleep
elapsed_seconds = 0
TEN_MINUTES = 60 * 10  # Make it longer for computation consuming training jobs
current_status = 0
while True:
    info = trainJobApi.view_training_job(project_id=PROJECT_ID, job_id=job_id, version_id=version_id)

    if current_status == info.status:
        sleep(5)
        continue
    else:
        current_status = info.status
    
    print(info.status)
    if info.status == 1:
        print('Job {}@{} initializing'.format(job_id, version_id))  
        
    elif info.status == 8:
        print('Job {}@{} running'.format(job_id, version_id))
    elif info.status == 10:
        print('Job {}@{} complete'.format(job_id, version_id))
        break
    elif info.status == 11:
        print('Job {}@{} failed'.format(job_id, version_id))
        print(info)
        break
        
    sleep(5)
    elapsed_seconds += 5
    if elapsed_seconds >= TEN_MINUTES:
        print('Job {}@{} timeout'.format(job_id, version_id))
        break
            

NameError: name 'job_id' is not defined

导入模型，上传config.json和customize_service.py到model目录下

In [10]:
resp = obsClient.putFile(TARGET_BUCKET, 'mnist/output/model/config.json', file_path='./config.json')
print('upload config.json for model:\n{}\n'.format(resp.body))
if resp.status >= 300:
    print('Failed to put file')  

resp = obsClient.putFile(TARGET_BUCKET, 'mnist/output/model/customize_service.py', file_path='./customize_service.py')
print('upload customize_service.py for model:\n{}\n'.format(resp.body))
if resp.status >= 300:
    print('Failed to put file')  
    

upload config.json for model:
{'versionId': 'G0011168E7168443FFFF801211230AD2', 'etag': '"bd6a5ac6acf304030075497d575669e1"', 'objectUrl': 'https://ma-workflow.obs.cn-north-1.myhuaweicloud.com/mnist/output/model/config.json'}

upload customize_service.py for model:
{'versionId': 'G0011168E7168486FFFF800E110247F6', 'etag': '"f242ba3a83d8127807ca7b2cdfd0fbcb"', 'objectUrl': 'https://ma-workflow.obs.cn-north-1.myhuaweicloud.com/mnist/output/model/customize_service.py'}



In [30]:
modelApi = ModelApi(maClient)

model_body = {
  "model_name": "mxnet-mnist-model",
  "model_version": "0.0.1",
#   source_location到model目录上一层即可
  "source_location": "https://ma-workflow.obs.cn-north-1.myhwclouds.com/mnist/output",
#   注释掉的参数为必填，但是从配置文件中读取
  "model_type": "MXNet",
  "model_algorithm": "image_classification",
#   "input_params": [
#     {
#       "url": "/object_detection",
#       "param_name": "field0",
#       "param_type": "int"
#     }
#   ],
#   "output_params": [
#     {
#       "url": "/object_detection",
#       "param_name": "res0",
#       "param_type": "int"
#     }
#   ],
#   "dependencies": [
#     {
#       "installer": "pip",
#       "packages": [
#         {
#           "package_name": "numpy",
#           "package_version": "1.5.0",
#           "restraint": "ATLEAST"
#         }
#       ]
#     }
#   ]
}
model_id = None
try:
    imported_model = modelApi.create_the_model(project_id=PROJECT_ID, body=model_body)
    model_id = imported_model.model_id
except ApiException as e:
    error_code = json.loads(e.body)['error_code']
    if error_code == "ModelArts.3002":
        print("Get the model_id")
        model_list = modelApi.get_model_list(project_id=PROJECT_ID)
        for model in model_list.models:
            if model.model_name == 'mxnet-mnist-model':
                model_id = model.model_id
                break

if model_id == None:
    raise(Exception('Cannot find the model ID'))

E:\Dev Tools\Anaconda3\envs\py36\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Get the model_id
model_id:8319ccbf-408f-494c-8925-93866727f6c9


E:\Dev Tools\Anaconda3\envs\py36\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [56]:
serviceApi = ServiceApi(maClient)

body = {
    "service_name": "mxnet-mnist-service",
    "description": "mxnet mnist service",
    "infer_type": "real-time",
    "config": [
        {
            "model_id": model_id,
            "weight": "100",
            "specification": "modelarts.vm.cpu.2u",
            "instance_count": 1,
            "envs": {
                "input_data_name": "images",
                "input_data_shape": "0,1,28,28",
                "output_data_shape": "0,10"
            }
        }
    ]
}
service_id = None

try:
    deployed_service = serviceApi.create_service(project_id=PROJECT_ID, body = body)
    service_id = deployed_service.service_id
except ApiException as e:
    error_code = json.loads(e.body)['error_code']
    if error_code == 'ModelArts.3503':
        service_list = serviceApi.get_service_list(project_id=PROJECT_ID)
        for svc in service_list.services:
            if svc.service_name == 'mxnet-mnist-service':
                service_id = svc.service_id
                break

if service_id == None:
    raise(Exception('service id not found!'))
    



E:\Dev Tools\Anaconda3\envs\py36\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [75]:
service_status = ''
info = None
# 服务状态，取值为running/deploying/concerning/failed/stopped/finished。
elapsed_seconds = 0
while True:
    info = serviceApi.get_service_info(project_id=PROJECT_ID, service_id=service_id)
    if service_status == info.status:
        sleep(5)
        continue
    else:
        service_status = info.status
    print('service status: {}'.format(service_status))
    
    if service_status == 'failed' or service_status == 'stopped' or service_status == 'finished' or service_status == 'running':
        break
    sleep(5)
    elapsed_seconds += 5
    if elapsed_seconds >= TEN_MINUTES:
        raise(Exception('timeout when deploying service {}'.format(service_id)))
        break

if service_status != 'running':
    raise(Exception('Invalid service status: ' + service_status))


E:\Dev Tools\Anaconda3\envs\py36\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


service status: running


In [74]:
# !pip install Pillow
# from make_your_mnist import generateValidationImage

# for i in range (0, 10):
#     generateValidationImage(str(i))

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


FileNotFoundError: [Errno 2] No such file or directory: './validation_images0.jpg'

In [76]:
print(info)

{'access_address': 'https://84604aaeecc9487aa252b8c9c7db8b10.apigw.cn-north-1.huaweicloud.com/v1/infers/dafb25e2-db6d-4fc5-a73a-6e22bd3816a3',
 'config': [{'envs': {'input_data_name': 'images',
                      'input_data_shape': '0,1,28,28',
                      'output_data_shape': '0,10'},
             'instance_count': 1,
             'model_id': '8319ccbf-408f-494c-8925-93866727f6c9',
             'model_name': 'mxnet-mnist-model',
             'model_version': '0.0.1',
             'scaling': False,
             'source_type': None,
             'specification': 'modelarts.vm.cpu.2u',
             'status': 'ready',
             'weight': '100'}],
 'description': 'mxnet mnist service',
 'error_msg': None,
 'failed_times': 0,
 'infer_type': 'real-time',
 'invocation_times': 3,
 'is_shared': False,
 'owner': '99d3096355bc4c98a82093283c41ee5a',
 'progress': 100,
 'project': 'cf1bed56cdc045b196278cace96852a8',
 'publish_at': '1550068517915',
 'security_group_id': None,
 'servi

In [113]:
!pip install requests
import requests
xauth_body = { 
  "auth": { 
    "identity": { 
      "methods": [ 
        "password" 
      ], 
      "password": { 
        "user": { 
          "name": "crystaldust", 
          "password": "containerops123", 
          "domain": { 
            "name": "crystaldust" 
          } 
        } 
      } 
    }, 
    "scope": { 
      "project": { 
        "name": "cn-north-1"
      } 
    } 
  } 
}
with open('C:/Users/lance/.modelarts/config.json') as f:
#     print(f.read())
    obj = json.load(f)
    
    
    username = obj['users'][0]['user']['username']
    password = obj['users'][0]['user']['password']
    
    xauth_body['auth']['identity']['password']['user']['name'] = username
    xauth_body['auth']['identity']['password']['user']['password'] = password
    xauth_body['auth']['identity']['password']['user']['domain']['name'] = username

jsonbody = json.dumps(xauth_body)

resp = requests.post('https://iam.cn-north-1.myhuaweicloud.com/v3/auth/tokens', data=jsonbody)
token = resp.headers['X-Subject-Token']
if token == None:
    raise(Exception('Failed to get X-Subject-Token'))

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [118]:
files = {'images': open('G:/chrome-downloads/0.jpg', 'rb')}
headers = {'X-Auth-Token': token}
resp = requests.post(info.access_address, files=files, headers=headers)
print(resp.json())

[{'class': 0, 'probability': 1.0}, {'class': 1, 'probability': 0.0}, {'class': 2, 'probability': 0.0}, {'class': 3, 'probability': 0.0}, {'class': 4, 'probability': 0.0}]
